In [20]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
img_size = 48
batch_size = 32
epochs = 20
learning_rate = 3e-4
step_size = 5
gamma = 0.1
data_dir = '/content/drive/MyDrive/dataset_age'
save_path = '/content/drive/MyDrive/age.pth'

In [3]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

In [4]:
rgb_mean = [0.485, 0.456, 0.406]
rgb_std = [0.229, 0.224, 0.225]

In [5]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [6]:
train_tf = transforms.Compose([
    transforms.RandomResizedCrop(img_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(rgb_mean, rgb_std),
])
test_tf = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize(rgb_mean, rgb_std),
])

In [7]:
# data
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_tf)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=test_tf)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

classes = train_dataset.classes
num_classes = len(classes)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [8]:
# model
import timm
model = timm.create_model(
    'mobilevit_s',
    pretrained=True,
    num_classes=num_classes,
    in_chans=3
).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/22.4M [00:00<?, ?B/s]

In [9]:
# Loss, Optimizer, Scheduler
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-2)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

# train
best_acc = 0.0
for epoch in range(1, epochs + 1):
    model.train()
    running_loss = 0.0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
    scheduler.step()
    epoch_loss = running_loss / len(train_dataset)

    model.eval()
    correct = total = 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    test_acc = correct / total * 100
    print(f"Epoch {epoch:02d}: Train Loss={epoch_loss:.4f}, Test Acc={test_acc:.2f}%")
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), save_path)
        print(f" New best model saved with acc={best_acc:.2f}%")
print(f"Best Test Accuracy: {best_acc:.2f}%")

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 01: Train Loss=1.6913, Test Acc=43.53%
 New best model saved with acc=43.53%
Epoch 02: Train Loss=1.4363, Test Acc=44.75%
 New best model saved with acc=44.75%
Epoch 03: Train Loss=1.3739, Test Acc=47.74%
 New best model saved with acc=47.74%
Epoch 04: Train Loss=1.3296, Test Acc=47.53%
Epoch 05: Train Loss=1.3011, Test Acc=49.21%
 New best model saved with acc=49.21%
Epoch 06: Train Loss=1.2458, Test Acc=50.50%
 New best model saved with acc=50.50%
Epoch 07: Train Loss=1.2150, Test Acc=51.04%
 New best model saved with acc=51.04%
Epoch 08: Train Loss=1.2102, Test Acc=50.44%
Epoch 09: Train Loss=1.2006, Test Acc=50.87%
Epoch 10: Train Loss=1.2028, Test Acc=51.78%
 New best model saved with acc=51.78%
Epoch 11: Train Loss=1.1834, Test Acc=51.62%
Epoch 12: Train Loss=1.1863, Test Acc=51.39%
Epoch 13: Train Loss=1.1903, Test Acc=51.68%
Epoch 14: Train Loss=1.1928, Test Acc=51.20%
Epoch 15: Train Loss=1.1890, Test Acc=51.68%
Epoch 16: Train Loss=1.1845, Test Acc=51.35%
Epoch 17: Trai

In [21]:
# predict
from PIL import Image
model.load_state_dict(torch.load(save_path, map_location=device))
model.eval()

def predict(img_path):
    img = Image.open(img_path).convert('RGB')
    x = test_tf(img).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(x)
    idx = logits.argmax(dim=1).item()
    return classes[idx]

pred = predict('/content/drive/MyDrive/iu.jpg')
print('Predicted race:', pred)

Predicted race: 20~29
